In [2]:
import re
from collections import defaultdict, Counter
from itertools import groupby
from nltk import flatten
from nltk.util import trigrams as get_trigrams
from mwu_measures.preprocessing_corpus import clean_bnc_line
import mwu_measures
# from mwu_measures.processing_corpus import Corpus


In [3]:
import duckdb

In [4]:
corpus_conn = duckdb.connect(":memory:")
corpus_conn.execute("""
    CREATE TABLE trigram_db (
             corpus TEXT,
             ug_1 TEXT,
             ug_2 TEXT,
             ug_3 TEXT,
             freq INTEGER
             )
""")
corpus_conn.execute("""
    CREATE TABLE unigram_db (
             corpus TEXT,
             ug TEXT,
             freq INTEGER
             )
""")


In [5]:
# chunk_trigrams = pd.DataFrame(chunk_trigrams, columns=['corpus', 'ug_1', 'ug_2', 'ug_3', 'freq'])

In [6]:
import pandas as pd
# with open('small_corpus.txt', 'r', encoding="utf-8") as corpus_file:
#     i = 0
#     while True:
#         raw_lines = corpus_file.readlines(100000)
#         if not raw_lines:
#         # if i == 1:
#             break
#         chunk_trigrams = mwu_measures.preprocessing_corpus.preprocess_bnc(raw_lines)
#         chunk_trigrams = pd.DataFrame(chunk_trigrams, columns=['corpus', 'ug_1', 'ug_2', 'ug_3', 'freq'])
#     # this_corpus.add_chunk(ngram_dicts)
#     # if verbose:
#         corpus_conn.register('chunk_df', chunk_trigrams)
#         corpus_conn.execute("INSERT INTO corpus_db SELECT * FROM chunk_df")
#         corpus_conn.execute("""
#         CREATE OR REPLACE TABLE corpus_db AS
#         SELECT corpus, ug_1, ug_2, ug_3, SUM(freq) AS freq
#         FROM corpus_db
#         GROUP BY corpus, ug_1, ug_2, ug_3
#                      """)
#         # corpus_conn.executemany("INSERT INTO corpus_db VALUES (?, ?, ?, ?, ?)", chunk_trigrams)
#         i += len(raw_lines)
#         print(f'{i} lines processed')

In [7]:
with open('small_corpus.txt', 'r', encoding="utf-8") as corpus_file:
    i = 0
    while True:
        raw_lines = corpus_file.readlines(1000000)
        if not raw_lines:
            break
        chunk_unigrams, chunk_trigrams = mwu_measures.preprocessing_corpus.preprocess_bnc(raw_lines)
        chunk_unigrams = pd.DataFrame(chunk_unigrams, columns=['corpus', 'ug', 'freq'])
        chunk_trigrams = pd.DataFrame(chunk_trigrams, columns=['corpus', 'ug_1', 'ug_2', 'ug_3', 'freq'])
    # if verbose:
        corpus_conn.register('chunk_unigrams', chunk_unigrams)
        corpus_conn.execute("INSERT INTO unigram_db SELECT * FROM chunk_unigrams")
        corpus_conn.register('chunk_trigrams', chunk_trigrams)
        corpus_conn.execute("INSERT INTO trigram_db SELECT * FROM chunk_trigrams")
        i += len(raw_lines)
        print(f'{i} lines processed')
corpus_conn.execute("""
CREATE OR REPLACE TABLE trigram_db AS
SELECT corpus, ug_1, ug_2, ug_3, SUM(freq) AS freq
FROM trigram_db
GROUP BY corpus, ug_1, ug_2, ug_3
                """)
corpus_conn.execute("""
CREATE OR REPLACE TABLE unigram_db AS
SELECT corpus, ug, SUM(freq) AS freq
FROM unigram_db
GROUP BY corpus, ug
                """)

8869 lines processed
18061 lines processed
26143 lines processed
35855 lines processed
44668 lines processed
52595 lines processed
59068 lines processed
66339 lines processed
74464 lines processed
82889 lines processed
92328 lines processed
100813 lines processed
108896 lines processed
119030 lines processed
128921 lines processed
140106 lines processed
148866 lines processed
156742 lines processed
164632 lines processed
172274 lines processed
180710 lines processed
189442 lines processed
196814 lines processed
207307 lines processed
215038 lines processed
224376 lines processed
231505 lines processed
240727 lines processed
250136 lines processed
261922 lines processed
269185 lines processed
275898 lines processed
287977 lines processed
300348 lines processed
308920 lines processed
318026 lines processed
328876 lines processed
339569 lines processed
349336 lines processed
357562 lines processed
366520 lines processed
376116 lines processed
386292 lines processed
395251 lines processed


In [7]:
x = corpus_conn.execute("SELECT corpus, ug_1, ug_2, ug_3, freq FROM corpus_db").fetchall()
y = corpus_conn.execute("SELECT corpus, ug_1, ug_2, ug_3, SUM(freq) FROM corpus_db GROUP BY corpus, ug_1, ug_2, ug_3").fetchall()

In [9]:
# fetch distributions for trigrams
corpus_conn.execute("SELECT * FROM trigram_db WHERE ug_1 = 'in' AND ug_2 = 'the'").fetch_df()


,corpus,ug_1,ug_2,ug_3,freq
0,C,in,the,hull,1.0
1,C,in,the,form,148.0
2,C,in,the,office,24.0
3,C,in,the,bad,5.0
4,C,in,the,evening,38.0
...,...,...,...,...,...
35285,A,in,the,exploitation,1.0
35286,A,in,the,rigging,1.0
35287,A,in,the,lions,1.0
35288,A,in,the,surf,1.0


In [10]:
# fetch distributions for bigrams
corpus_conn.execute("SELECT corpus, ug_1, ug_2, SUM(freq) AS freq FROM trigram_db WHERE ug_1 = 'in' GROUP BY corpus, ug_1, ug_2").fetch_df()


,corpus,ug_1,ug_2,freq
0,C,in,earlier,14.0
1,C,in,meals,1.0
2,C,in,an,782.0
3,C,in,eighth,1.0
4,C,in,fact,663.0
...,...,...,...,...
51350,E,in,kotah,1.0
51351,H,in,coffee,1.0
51352,F,in,participating,1.0
51353,F,in,terraced,1.0


In [11]:
# fetch distribution for unigrams
corpus_conn.execute("SELECT corpus, ug, freq  FROM unigram_db WHERE ug = 'in'").fetch_df()

,corpus,ug,freq
0,G,in,39689.0
1,B,in,34589.0
2,D,in,313.0
3,K,in,48643.0
4,E,in,36498.0
5,C,in,82781.0
6,J,in,26234.0
7,H,in,101224.0
8,F,in,45324.0
9,A,in,70156.0


In [12]:
# corpus proportions
corpus_conn.execute("SELECT corpus, SUM(freq) / (SELECT SUM(freq) FROM unigram_db) AS freq FROM unigram_db GROUP BY corpus").fetch_df()

,corpus,freq
0,A,0.141721
1,B,0.067463
2,C,0.175689
3,D,0.000864
4,E,0.067932
5,F,0.090955
6,G,0.078808
7,H,0.199364
8,J,0.059024
9,K,0.118179


In [14]:
# total_ngrams
corpus_conn.execute("SELECT SUM(freq) FROM trigram_db").fetchall()

[(21563350,)]

In [ ]:
# total_unigrams
corpus_conn.execute("SELECT ug, SUM(freq) AS freq FROM unigram_db GROUP BY ug").fetch_df()


,ug,freq
0,discomfiting,3.0
1,milelong,4.0
2,forsythia,6.0
3,boater,8.0
4,preen,6.0
...,...,...
310261,symbolisation,1.0
310262,underinformed,1.0
310263,trexacarna,1.0
310264,onetoothapiece,1.0


In [18]:
# total_trigrams
corpus_conn.execute("SELECT ug_1, ug_2, ug_3, SUM(freq) AS freq FROM trigram_db GROUP BY ug_1, ug_2, ug_3").fetch_df()

# Compare old procedure with DuckDB numbers. Unigram frequencies? Bigram frequencies? Trigram frequencies? Are they the same?

,ug_1,ug_2,ug_3,freq
0,it,now,i,12.0
1,streets,and,i,1.0
2,hugh,had,said,1.0
3,several,customers,at,1.0
4,an,italian,well,1.0
...,...,...,...,...
12676967,be,held,fast,1.0
12676968,distance,behind,them,1.0
12676969,bravado,where,i,1.0
12676970,to,shhh,and,1.0


In [26]:
# type frequencies
#bigrams
corpus_conn.execute("SELECT ug_1, COUNT(DISTINCT(ug_2)) AS type_freq FROM trigram_db GROUP BY ug_1").fetch_df()
corpus_conn.execute("SELECT ug_2, COUNT(DISTINCT(ug_1)) AS type_freq FROM trigram_db GROUP BY ug_2").fetch_df()

,ug_2,type_freq
0,is,24398
1,buttresses,14
2,seats,260
3,at,17874
4,only,4427
...,...,...
284831,l'instauration,1
284832,micheson,1
284833,taylormorgan,1
284834,bayu,1


In [27]:
# trigram
corpus_conn.execute("SELECT ug_1, ug_2, COUNT(DISTINCT(ug_3)) AS type_freq FROM trigram_db GROUP BY ug_1, ug_2").fetch_df()
corpus_conn.execute("SELECT ug_3, COUNT(DISTINCT(ug_1, ug_2)) AS type_freq FROM trigram_db GROUP BY ug_3").fetch_df()

,ug_3,type_freq
0,the,383891
1,them,20337
2,and,358892
3,effects,1601
4,overall,1005
...,...,...
289487,deinonychosaurs,1
289488,10.67,1
289489,arubano,1
289490,charolais.there's,1


In [ ]:
this_chunk = {corpus: Counter(trigrams) for corpus, trigrams in ngram_dicts[1].items()}
this_chunk = [(corpus, ngram[0], ngram[1], ngram[2], freq) for corpus, corpus_dict in this_chunk.items() for ngram, freq in corpus_dict.items()]

In [ ]:
corpus_db.execute("""
    CREATE TABLE corpus_db (
             corpus TEXT,
             ug_1 TEXT,
             ug_2 TEXT,
             ug_3 TEXT,
             freq INTEGER
             )
""")

In [ ]:
corpus_db.executemany("INSERT INTO corpus_db VALUES (?, ?, ?, ?, ?)", this_chunk)

In [ ]:
corpus_db.execute("SELECT * FROM corpus_db").fetchall()

In [ ]:
this_corpus = mwu_measures.processing_corpus.process_corpus(corpus_name='bnc', corpus_dir='small_corpus.txt', verbose=False, chunk_size=10000)

In [ ]:
# import pandas as pd
# mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
# mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
# mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
# mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
# print(f'Number of possible bigrams and trigrams: {len(mwu_examples)}')

In [ ]:
# ngrams = mwu_examples.sample(100).Expression.to_list()

In [ ]:
# occurring_unigrams = flatten([ngram.split() for ngram in ngrams])

In [ ]:
# for corpus, trigram_list in this_corpus.trigram_freqs.items():
#     for trigram, frequency in trigram_list.items():
#         if any(unigram in occurring_unigrams for unigram in trigram):
#             print(trigram)

In [ ]:
# reduced_trigrams = {corpus: {trigram: frequency for trigram, frequency in corpus_dict.items() if any(unigram in occurring_unigrams for unigram in trigram)} for corpus, corpus_dict in this_corpus.trigram_freqs.items()}

In [ ]:
# org_items = groupby(raw_lines, key=lambda x: re.match(r'(^.)', x).group(1))
# unigrams = {key:[clean_bnc_line(line) for line in group] for key, group in org_items
#             }


In [ ]:
# trigrams = {
#     key:[bigram for line in group for bigram in get_trigrams(line) if len(line) > 1]
#     for key, group in unigrams.items()
#     }
# unigrams = {corpus: flatten(units) for corpus, units in unigrams.items()}
# ngram_dicts = (unigrams, trigrams)

In [ ]:
import pandas as pd
# x = pd.DataFrame(this_corpus.trigram_freqs)

In [ ]:
x = {}
for corpus, freq_list in this_corpus.trigram_freqs.items():    
    this_index = pd.MultiIndex.from_tuples(freq_list.keys(), names = ['first', 'second', 'third'])
    freqs = pd.Series(freq_list.values(), index=this_index).sort_index()
    x[corpus] = freqs

In [ ]:
from pympler import asizeof
print(asizeof.asizeof(this_corpus.trigram_freqs))
print(asizeof.asizeof(x))

In [ ]:
print(this_corpus.trigram_freqs['C'][('most', 'of', 'the')])
print(x['C']['most', 'of'])

In [ ]:
clean_list = [unigram for unigram, freq in this_corpus.unigram_total.items() if freq < 3]

In [ ]:
for trigram in this_corpus.trigram_freqs['C'].keys():
    # print(trigram)
    if any(map(lambda x: x in clean_list, trigram)):
        print(trigram)
        


In [ ]:
len(this_corpus.unigram_total) /  len(x)

In [ ]:
# print(this_corpus.get_ngram('was'))
print(this_corpus.query_ngram('was built')) # this one's wrong, should be built: 5
print(this_corpus.query_ngram('was built in'))
print(this_corpus.query_ngram('of the')) # this one's wrong, should be built: 5


In [ ]:
# print(this_corpus.get_ngram('was'))
print(this_corpus.query_inverse_ngram('was built in'))
print(this_corpus.query_inverse_ngram('of the'))

In [ ]:
this_corpus.n_trigrams